In [10]:
import pandas as pd
import numpy as np
from sklearn import * 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [8]:
df = pd.read_csv("/data/kaggle/house-prices/data_combined_cleaned.csv")
del df["Id"]
df = df[~np.isnan(df.SalesPrice)]
df.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [12]:
target = "SalesPrice"
y = np.log(df[target])
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                            test_size = 0.3, random_state = 1)
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LinearRegression())
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("Training MSE", metrics.mean_squared_error(y_train, y_train_pred),
     "\nTesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
     "\nTraining R2", metrics.r2_score(y_train, y_train_pred),
     "\nTesting R2", metrics.r2_score(y_test, y_test_pred))

Training MSE 6.996007534795658e-29 
Testing MSE 0.038670365461414107 
Training R2 1.0 
Testing R2 0.7953835156551845


In [34]:
target = "SalesPrice"
y = np.log(df[target])
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                            test_size = 0.3, random_state = 1)
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01))
])

pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("Training MSE", metrics.mean_squared_error(y_train, y_train_pred),
     "\nTesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
     "\nTraining R2", metrics.r2_score(y_train, y_train_pred),
     "\nTesting R2", metrics.r2_score(y_test, y_test_pred))

Training MSE 0.01552703945002047 
Testing MSE 0.022954852934347732 
Training R2 0.8940480673803641 
Testing R2 0.8785389987905549


In [32]:
weights = pd.DataFrame({"feature": X.columns, "coef": pipe.steps[2][1].coef_})
weights["abs_coef"] = np.abs(weights.coef)
weights = weights.sort_values("abs_coef", ascending = False)
del weights["abs_coef"]
weights

,coef,feature
110,3.417390e-01,RoofMatl_CompShg
114,2.426316e-01,RoofMatl_Tar&Grv
116,1.565532e-01,RoofMatl_WdShngl
115,1.260482e-01,RoofMatl_WdShake
38,1.022773e-01,MSZoning_RL
13,9.244006e-02,2ndFlrSF
226,-9.227119e-02,GarageQual_TA
111,8.846087e-02,RoofMatl_Membran
12,8.197077e-02,1stFlrSF
39,7.998174e-02,MSZoning_RM


In [ ]:
.df.sort_values